# For those who want to learn quickly:)

* **This is the dataset I downloaded at the beginning of the competition. I want to use this dataset. Because the notebooks I want to compare are the ones published at the beginning of the competition.**
* **I took the dates after 2021 to compare the sharpe ratio in the reference notebook.**

**link** : https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164&cellId=54

* **I have used AdjustedClose, It comes from this notebook:**

**link** https://www.kaggle.com/code/smeitoma/train-demo?scriptVersionId=92137850&cellId=6

* **Cal_sharpe_ratio(train_df) with actual Target**
5.790745682567137

**link**:https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164&cellId=60

* **Cal_sharpe_ratio(train_df) with actual Target and my code**
5.790745682567137

* **Cal_sharpe_ratio(train_df) with Calc. Target and my code**
5.7907409610656

* **Cal_sharpe_ratio(supplemental_df)**
5.434704919895065



# Read Dataset

**WARNING!**

**This is the dataset I downloaded at the beginning of the competition. I want to use this dataset. Because the notebooks I want to compare are the ones published at the beginning of the competition.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('../input/jpx-tokyo-dataset/train_files/stock_prices.csv')

# Calculation AdjustedClose

**It comes from this notebook:**

https://www.kaggle.com/code/smeitoma/train-demo?scriptVersionId=92137850&cellId=6

In [ ]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

# Constraints

In [ ]:
df = adjust_price(df)
df = df.loc[df.index>= "2021-01-01"]


**I took the dates after 2021 to compare the sharpe ratio in the reference notebook.**

**Reference link** : https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164&cellId=54

In [ ]:
AdjustedClose = df.pivot(columns='SecuritiesCode', values='AdjustedClose')
AdjustedClose.head()

# Calculation of Target

**Target calculation with shifting 2 days for all SecuritiesCodes**

In [ ]:
# target calculation with shifting 2 days for all SecuritiesCodes
Cal_Target = (AdjustedClose.shift(-2) - AdjustedClose.shift(-1)).div(AdjustedClose.shift(-1))

# we can directly add last two days from df for because we shift 2 days 
Cal_Target[Cal_Target.index=="2021-12-02"] = df[df.index=="2021-12-02"]["Target"].values
Cal_Target[Cal_Target.index=="2021-12-03"] = df[df.index=="2021-12-03"]["Target"].values
Cal_Target.head()

**Let's get actual values of Target to compare**

In [ ]:
Target = df.pivot(columns='SecuritiesCode', values='Target')
Target.head()

**Target and Cal_Target looks same but we need to check!**

In [ ]:
Target==Cal_Target

**For ex. values of 1301**


In [ ]:
print(Target[1301][1],Cal_Target[1301][1])

**Target and Cal_Target don't have same values. Don't worry, Let's Continue calculating sharpe ratio**.
**Get first and last 200 values of Cal_Target, and calculate sharpe ratio from daily_spread_returns**

In [ ]:
first200=pd.DataFrame(-np.sort(-Cal_Target.values)).loc[:,0:199]
last200=pd.DataFrame(np.sort(Cal_Target.values)).loc[:,0:199]

weights = np.linspace(start=2, stop=1, num=200)

Sup=((first200 * weights).sum(axis = 1))/np.mean(weights)
Sdown=((last200 * weights).sum(axis = 1))/np.mean(weights)

**Calculate daily spread returns for every dates**

In [ ]:
daily_spread_returns = (Sup - Sdown)
daily_spread_returns

In [ ]:
sharpe_ratio = daily_spread_returns.mean()/daily_spread_returns.std()
sharpe_ratio

* **sharpe_ratio = 5.790745682567136**

**sharpe_ratio of Cal_Target and Target are slightly different**

**reference link**:https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164&cellId=60

**Let's write efficient code,
we should use actual Target**

In [ ]:
import pandas as pd

train_df = pd.read_csv('../input/jpx-tokyo-dataset/train_files/stock_prices.csv')
train_df = train_df.loc[train_df.Date>= "2021-01-01"]

supplemental_df  = pd.read_csv("../input/jpx-tokyo-dataset/supplemental_files/stock_prices.csv")

In [ ]:
def Cal_sharpe_ratio(df):
    
    def first200(df):
          return pd.DataFrame(-np.sort(-df["Target"])[:200]).T
    first200 = df.groupby('Date').apply(first200)

    def last200(df):
          return pd.DataFrame(np.sort(df["Target"])[0:200]).T
    last200 = df.groupby('Date').apply(last200)

    weights = np.linspace(start=2, stop=1, num=200)

    Sup=((first200 * weights).sum(axis = 1))/np.mean(weights)
    Sdown=((last200 * weights).sum(axis = 1))/np.mean(weights)
    
    daily_spread_returns = (Sup - Sdown)
    sharpe_ratio = (Sup - Sdown).mean()/(Sup - Sdown).std()
    
    return sharpe_ratio


**sharpe_ratio calculation for supplemental df**

In [ ]:
Cal_sharpe_ratio(supplemental_df)

**sharpe_ratio calculation for train df**

In [ ]:
Cal_sharpe_ratio(train_df)

* **I reach same value with my code**
* **sharpe ratio of train_df => link** : https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164&cellId=60